In [1]:
import geopandas as gpd
import pandas as pd
import arcpy
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

In [2]:
path = 'C:\projects\Mars_weather'

In [3]:
df = pd.read_csv(path + '\df.csv')
df = df.drop(columns = 'Unnamed: 0')

In [4]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.lattitude), crs='EPSG:4326')

In [5]:
gdf.head()

,ind,x,y,z,lattitude,longitude,atm_pressure,temperature,zonal_wind,meridional_wind,height,CO2,speed_wind,direction_wind,file_number,date,geometry
0,0,-0.850651,0.525731,0.000000,0.000000,-58.282501,537.609497,-73.270102,-1.038340,0.048111,1020.968140,96.38,1.039454,357.347124,1,2021-01-01,POINT (-58.28250 0.00000)
1,1,0.850651,-0.525731,0.000000,0.000000,121.717003,572.585266,-73.694266,-1.378017,0.146964,333.579285,96.35,1.385832,353.912478,1,2021-01-01,POINT (121.71700 0.00000)
2,2,-0.850651,-0.525731,0.000000,0.000000,-121.717003,383.006866,-98.963583,-7.052595,6.349962,4859.559082,96.36,9.490053,318.001001,1,2021-01-01,POINT (-121.71700 0.00000)
3,3,-0.510382,0.052621,0.858337,59.130299,-84.113503,775.726196,-98.672263,6.694823,0.183575,-3258.103760,96.57,6.697339,181.570682,1,2021-01-01,POINT (-84.11350 59.13030)
4,4,-0.461270,0.053213,0.885663,62.333199,-83.419296,802.746765,-104.572943,7.749964,-0.290330,-3641.076904,96.68,7.755400,177.854582,1,2021-01-01,POINT (-83.41930 62.33320)


In [6]:
gdf.to_file(path+r'\huge_file.shp')

In [7]:
#splitting 1 file into many small ones
arcpy.analysis.SplitByAttributes(path+r'\huge_file.shp', path+r'\files_shp', "file_number")

In [8]:
file_number = df['file_number'][-1:].item()

In [9]:
#conversion to rasters
for i in range(1, file_number):
    out_raster_tem = arcpy.sa.Idw(path+r'\files_shp'+'\\'+str(i)+'_0.shp','temperature', 0.76, 2)
out_raster_tem.save(path + r"\rastry_temp2\tem"+str(i-1))

In [10]:
out_raster_atm = arcpy.sa.Idw(path+r'\files_shp'+'\\'+str(i)+'_0.shp','atm_pressure', 0.76, 2)
out_raster_atm.save(path + r"\rastry_atm\atm_"+str(i-1))

out_raster_co2 = arcpy.sa.Idw(path+r'\files_shp'+'\\'+str(i)+'_0.shp', 'CO2',0.76, 2)
out_raster_co2.save(path + r"\rastry_CO2\co2_"+str(i-1))

In [11]:
#determine min, max
min_atm = min(gdf['atm_pressure'])
max_atm = max(gdf['atm_pressure'])

min_tem = min(gdf['temperature'])
max_tem = max(gdf['temperature'])

min_co2 = min(gdf['CO2'])
max_co2 = max(gdf['CO2'])

In [12]:
#create list to Reclassify
dif_tem = (max_tem - min_tem)/20
dif_co2 = (max_co2 - min_co2)/20
dif_atm = (max_atm - min_atm)/20

list_tem = []
list_co2 = []
list_atm = []

In [13]:
for i in range (20):
    list_tem.append([min_tem+dif_tem*i, min_tem+dif_tem*(i+1), i+1])
    list_co2.append([min_co2+dif_co2*i, min_co2+dif_co2*(i+1), i+1])
    list_atm.append([min_atm+dif_atm*i, min_atm+dif_atm*(i+1), i+1])

In [14]:
#another way to list for CO2
co = gdf['CO2']
list_co2 = []
for i in range(20):
    list_co2.append([np.quantile(co, (i)/20), np.quantile(co, (i+1)/20), i+1])

In [15]:
#Reclassify
d_rec_tem = {}
d_rec_co2 = {}
d_rec_atm = {}
for i in range(file_number-1):
    d_rec_tem['rec_tem'+str(i)] = arcpy.sa.Reclassify(path + r'\rastry_temp\tem'+ str(i), "VALUE", arcpy.sa.RemapRange(list_tem), "NODATA")
    d_rec_tem['rec_tem'+str(i)].save(path+r'\reclassify_tem\rec_tem'+str(i))
    d_rec_co2['rec_co2_'+str(i)] = arcpy.sa.Reclassify(path + r'\rastry_CO2\co2_'+ str(i), "VALUE", arcpy.sa.RemapRange(list_co2), "NODATA")
    d_rec_co2['rec_co2_'+str(i)].save(path+r'\reclassify_CO2\rec_co2_'+str(i))
    d_rec_atm['rec_atm'+str(i)] = arcpy.sa.Reclassify(path + r'\rastry_atm\atm'+ str(i), "VALUE", arcpy.sa.RemapRange(list_arm), "NODATA")

In [16]:
#conversion to polygons
for i in range(file_number-1):
    arcpy.conversion.RasterToPolygon(path+r'\reclassify_tem\rec_tem'+str(i),path+r'\rastryPol_tem\rasPol_tem'+str(i)+'.shp',"SIMPLIFY", "VALUE", "SINGLE_OUTER_PART", None)
    arcpy.conversion.RasterToPolygon(path+r'\reclassify_CO2\rec_co2_'+str(i),path+r'\rastryPol_CO2\rasPol_co2_'+str(i)+'.shp',"SIMPLIFY", "VALUE", "SINGLE_OUTER_PART", None)
    arcpy.conversion.RasterToPolygon(path+r'\reclassify_atm\rec_atm'+str(i),path+r'\rastryPol_atm\rasPol_atm'+str(i)+'.shp',"SIMPLIFY", "VALUE", "SINGLE_OUTER_PART", None)

In [17]:
#adding a date column
for i in range(file_number-1):
    arcpy.management.AddField(path+r'\rastryPol_tem\rasPol_tem'+str(i)+'.shp',"date", "date", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
    arcpy.management.AddField(path+r'\rastryPol_CO2\rasPol_co2_'+str(i)+'.shp',"date", "date", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
    arcpy.management.AddField(path+r'\rastryPol_CO2v2\rasPol_co2_'+str(i)+'.shp',"date", "date", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
    arcpy.management.AddField(path+r'\rastryPol_atm\rasPol_atm'+str(i)+'.shp',"date", "date", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

In [18]:
#determining the value for column date
for i in range(file_number-1):
    d = datetime.datetime.strptime('2021 '+str(i+1), '%Y %j')
    data = d.strftime('%Y-%m-%d')
    data = "'"+data+"'"
    arcpy.management.CalculateField(path+r'\rastryPol_tem\rasPol_tem'+str(i)+'.shp', "date", data, "PYTHON3", '', "TEXT")
    arcpy.management.CalculateField(path+r'\rastryPol_CO2\rasPol_co2_'+str(i)+'.shp', "date", data, "PYTHON3", '', "TEXT")
    arcpy.management.CalculateField(path+r'\rastryPol_CO2v2\rasPol_co2_'+str(i)+'.shp', "date", data, "PYTHON3", '', "TEXT")
    arcpy.management.CalculateField(path+r'\rastryPol_atm\rasPol_atm'+str(i)+'.shp', "date", data, "PYTHON3", '', "TEXT")

In [19]:
#values to functions Merge
files_tem = ''
files_CO2 = ''
files_CO2v2 = ''
files_atm = ''
for i in range(file_number-1):
    files_tem = files_tem + path+r'\rastryPol_tem\rasPol_tem'+str(i)+'.shp'
    files_CO2 = files_CO2 + path+r'\rastryPol_CO2\rasPol_co2_'+str(i)+'.shp'
    files_CO2v2 = files_CO2v2 + path+r'\rastryPol_CO2v2\rasPol_co2_'+str(i)+'.shp'
    files_atm = files_atm + path+r'\rastryPol_atm\rasPol_atm'+str(i)+'.shp'
    if i != file_number-2:
        files_tem = files_tem + ';'
        files_CO2 = files_CO2 + ';'
        files_CO2v2 = files_CO2v2 + ';'
        files_atm = files_atm + ';'

In [20]:
for i in range(file_number-1):
    arcpy.management.Merge(files_tem, path+'\merge_files\merge_tem')
    arcpy.management.Merge(files_CO2, path+'\merge_files\merge_CO2')
    arcpy.management.Merge(files_CO2v2, path+'\merge_files\merge_CO2v2')
    arcpy.management.Merge(files_atm, path+'\merge_files\merge_atm')

In [21]:
#transformation of shp file into graphic files
for k in range(file_number-1):
    df = gpd.read_file(path+r"\rastryPol_oci\rasPol_oci"+str(k)+".shp")
    
    min_gr = min(df['gridcode'])
    max_gr = max(df['gridcode'])

    x = len(df)

    if min_gr != 1:
        for w in range(min_gr-1):
        df2 = {'Id': x+w+1, 'gridcode': min_gr-w-1}
        df = df.append(df2, ignore_index=True)

    x = len(df)

    if max_gr != 20:
        for w in range(20-max_gr):
            df2 = {'Id': x+w+1, 'gridcode': max_gr+w+1}
            df = df.append(df2, ignore_index=True)

    df.plot(column='gridcode', cmap='rainbow', edgecolor='grey', linewidth=0.5,figsize=(21, 11))

    plt.axis('off')
    plt.savefig(path+r'\png_files\file'+str(k)+'.png')

    img = cv.imread(path+r'\png_files\file'+str(k)+'.png')

    y = img.shape[0]
    x = img.shape[1]

    a = int(y / 2)

    for j in range(x):
        if (np.array_equal(img[a][j], [255, 255, 255])):
            if (j < x / 2):
                x_min = j
            elif (j > x / 2):
                x_max = j
                break

    b = int(x / 2)

    for i in range(y):
        if (np.array_equal(img[i][b], [255, 255, 255])):
            if (i < y / 2):
                y_min = i
            elif (i > y / 2):
                y_max = i
                break

    file = img[y_min:y_max, x_min:x_max]
    if k < 10:
        k = '00' + str(k)
    elif k < 100:
        k = '0' + str(k)
    else:
        k = str(k)

    cv.imwrite(path+r'\file'+str(k)+'.png', file)